In [ ]:
#From mnist example in pytorch example

In [2]:
from __future__ import print_function
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import torch.utils.data
import time

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.conv2_drop = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = x.view(-1, 320)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

In [4]:
def train( model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

def test( model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
            pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [5]:
device = torch.device("cpu")
batch_size = 64
#kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_dataset =  datasets.MNIST('../data', train=True, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
test_dataset =  datasets.MNIST('../data', train=False, transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))

model1_train = torch.utils.data.Subset(train_dataset,range(25000))
model2_train = torch.utils.data.Subset(train_dataset,range(25000,50000))
combine_train = torch.utils.data.Subset(train_dataset,range(50000,60000))

model1_train_loader = torch.utils.data.DataLoader(model1_train,batch_size=batch_size, shuffle=True)
model2_train_loader = torch.utils.data.DataLoader(model2_train,batch_size=batch_size, shuffle=True)
combine_train_loader = torch.utils.data.DataLoader(combine_train,batch_size=batch_size, shuffle=True)

train_loader = torch.utils.data.DataLoader(train_dataset,batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset,batch_size=batch_size, shuffle=True)


In [15]:
model = Net()

optimizer = optim.SGD(model.parameters(), lr=0.05)

single_model_start_time = time.time()
for epoch in range(1, 20):
    train( model, device, train_loader, optimizer, epoch)
    test( model, device, test_loader)
single_model_end_time = time.time()    

Train Epoch: 1 [0/60000 (0%)]	Loss: 2.376938
Train Epoch: 1 [6400/60000 (11%)]	Loss: 0.903542
Train Epoch: 1 [12800/60000 (21%)]	Loss: 0.632314
Train Epoch: 1 [19200/60000 (32%)]	Loss: 0.457679
Train Epoch: 1 [25600/60000 (43%)]	Loss: 0.443795
Train Epoch: 1 [32000/60000 (53%)]	Loss: 0.417793
Train Epoch: 1 [38400/60000 (64%)]	Loss: 0.360744
Train Epoch: 1 [44800/60000 (75%)]	Loss: 0.343218
Train Epoch: 1 [51200/60000 (85%)]	Loss: 0.316730
Train Epoch: 1 [57600/60000 (96%)]	Loss: 0.312192

Test set: Average loss: 0.1212, Accuracy: 9625/10000 (96%)

Train Epoch: 2 [0/60000 (0%)]	Loss: 0.309151
Train Epoch: 2 [6400/60000 (11%)]	Loss: 0.404300
Train Epoch: 2 [12800/60000 (21%)]	Loss: 0.493842
Train Epoch: 2 [19200/60000 (32%)]	Loss: 0.206859
Train Epoch: 2 [25600/60000 (43%)]	Loss: 0.271278
Train Epoch: 2 [32000/60000 (53%)]	Loss: 0.182909
Train Epoch: 2 [38400/60000 (64%)]	Loss: 0.306855
Train Epoch: 2 [44800/60000 (75%)]	Loss: 0.350212
Train Epoch: 2 [51200/60000 (85%)]	Loss: 0.255203
T

Train Epoch: 15 [51200/60000 (85%)]	Loss: 0.135983
Train Epoch: 15 [57600/60000 (96%)]	Loss: 0.051386

Test set: Average loss: 0.0324, Accuracy: 9897/10000 (99%)

Train Epoch: 16 [0/60000 (0%)]	Loss: 0.109488
Train Epoch: 16 [6400/60000 (11%)]	Loss: 0.008799
Train Epoch: 16 [12800/60000 (21%)]	Loss: 0.136251
Train Epoch: 16 [19200/60000 (32%)]	Loss: 0.060284
Train Epoch: 16 [25600/60000 (43%)]	Loss: 0.053560
Train Epoch: 16 [32000/60000 (53%)]	Loss: 0.061247
Train Epoch: 16 [38400/60000 (64%)]	Loss: 0.124663
Train Epoch: 16 [44800/60000 (75%)]	Loss: 0.382084
Train Epoch: 16 [51200/60000 (85%)]	Loss: 0.172176
Train Epoch: 16 [57600/60000 (96%)]	Loss: 0.085447

Test set: Average loss: 0.0332, Accuracy: 9893/10000 (99%)

Train Epoch: 17 [0/60000 (0%)]	Loss: 0.055696
Train Epoch: 17 [6400/60000 (11%)]	Loss: 0.262721
Train Epoch: 17 [12800/60000 (21%)]	Loss: 0.047206
Train Epoch: 17 [19200/60000 (32%)]	Loss: 0.093576
Train Epoch: 17 [25600/60000 (43%)]	Loss: 0.056773
Train Epoch: 17 [32000/

In [ ]:
#train both models separately on differnt datasets
model1 = Net()
model2 = Net()

optimizer1 = optim.SGD(model1.parameters(), lr=0.05)

model1_start_time = time.time()
for epoch in range(1, 20):
    train( model1, device, model1_train_loader, optimizer1, epoch)
    test( model1, device, test_loader)
model1_end_time = time.time()

optimizer2 = optim.SGD(model2.parameters(), lr=0.05)
model2_start_time = time.time()
for epoch in range(1, 20):
    train( model2, device, model2_train_loader, optimizer2, epoch)
    test( model2, device, test_loader)   
model2_end_time = time.time()    

In [14]:
#Combined training
combine_train_start_time = time.time()
for epoch in range(1, 20):
    model1.train()
    model2.train()
    for batch_idx, (data, target) in enumerate(combine_train_loader):
        data, target = data.to(device), target.to(device)
        optimizer1.zero_grad()
        optimizer2.zero_grad()
        output1 = model1(data)
        output2 = model2(data)
        pred1 = output1.max(1, keepdim=True)[1]
        pred2 = output2.max(1, keepdim=True)[1]
        loss1 = F.nll_loss(output1, pred2.view(-1))
        loss1.backward()
        optimizer1.step()
        loss2 = F.nll_loss(output2, pred1.view(-1))
        loss2.backward()
        optimizer2.step()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(combine_train_loader.dataset),
                100. * batch_idx / len(combine_train_loader), loss1.item(), loss2.item))
combine_train_end_time = time.time()            

Train Epoch: 1 [0/10000 (0%)]	Loss: 0.388519
Train Epoch: 1 [6400/10000 (64%)]	Loss: 0.438580
Train Epoch: 2 [0/10000 (0%)]	Loss: 0.444665
Train Epoch: 2 [6400/10000 (64%)]	Loss: 0.279676
Train Epoch: 3 [0/10000 (0%)]	Loss: 0.144020
Train Epoch: 3 [6400/10000 (64%)]	Loss: 0.525121
Train Epoch: 4 [0/10000 (0%)]	Loss: 0.310531
Train Epoch: 4 [6400/10000 (64%)]	Loss: 0.403705
Train Epoch: 5 [0/10000 (0%)]	Loss: 0.315960
Train Epoch: 5 [6400/10000 (64%)]	Loss: 0.324131
Train Epoch: 6 [0/10000 (0%)]	Loss: 0.426520
Train Epoch: 6 [6400/10000 (64%)]	Loss: 0.192550
Train Epoch: 7 [0/10000 (0%)]	Loss: 0.347938
Train Epoch: 7 [6400/10000 (64%)]	Loss: 0.424191
Train Epoch: 8 [0/10000 (0%)]	Loss: 0.562397
Train Epoch: 8 [6400/10000 (64%)]	Loss: 0.309457
Train Epoch: 9 [0/10000 (0%)]	Loss: 0.157770
Train Epoch: 9 [6400/10000 (64%)]	Loss: 0.591936
Train Epoch: 10 [0/10000 (0%)]	Loss: 0.573923
Train Epoch: 10 [6400/10000 (64%)]	Loss: 0.202599
Train Epoch: 11 [0/10000 (0%)]	Loss: 0.482513
Train Epoch:

In [16]:
single_model_end_time-single_model_start_time

839.3560137748718

In [21]:
model1_end_time-model1_start_time + combine_train_end_time -combine_train_start_time

705.5945563316345

In [19]:
test( model2, device, test_loader) 
test( model1, device, test_loader) 


Test set: Average loss: 0.0736, Accuracy: 9776/10000 (98%)


Test set: Average loss: 0.0735, Accuracy: 9772/10000 (98%)

